Fix a subgroup inclusion $H\leq G$. The function ``print_symbolic_norm`` displays formulas for the coefficients for the norm
$$ 
    \text{nm}_H^G \left( \sum a_i \cdot [H/K_i] \right) \in A(G)
$$
as polynomials in the coefficients $a_i$.

In [ ]:
def print_symbolic_norm(H,G):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    nG = matG.dimensions()[0] # dimensions of the table of marks
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    nH = matH.dimensions()[0] # dimension of the table of marks
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes
    
    R = PolynomialRing(QQ, nH, "a") # polynomial ring with nH many generators
    a = vector(R, R.gens()) # vector over symbolic ring R

    marksa = a * matH # convert the vector f of coefficients into an element of the ghost ring
    nm = vector(R, [1]*nG) # empty product is 1; setup for looping
    for i in range(nG):
        K = repsG[i] # we're finding the component indexed by K
        for(g,_) in G.DoubleCosetRepsAndSizes(K,H): # only need the reps of double cosets
            HcapKg = gap.Intersection(H,K.ConjugateGroup(g)) # form the intersection of H and the conjugate of K by g
            index = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, HcapKg)) # find the index of the subgroup of H that represents H \cap K^g
            nm[i] *= marksa[index] # multiply the [K] component of the norm by marksf([H \cap K^g])
    nm = matG.solve_left(nm) # convert back into a list of coefficients
    
    print("============ Norm from", gap.StructureDescription(H), "to", gap.StructureDescription(G), "============\n")    
    print("Norm of\n")

    for i in range(nH):
        if i == 0:
            print('\t',end='')
        print(a[i]," "+str(gap.StructureDescription(H)),"/",str(gap.StructureDescription(repsH[i])),sep='',end='')
        if i < nH-1:
            print(" + ",end='')
    
    print("\n\nhas coefficients:\n")
        
    for i in range(nG):
        print("coefficient of " + str(gap.StructureDescription(G)) + "/" + str(gap.StructureDescription(repsG[i])) + ":",end='')
        print("\t",factor(nm[i]))
        # uncomment this line if you want latex output
        # long term we should probably add a ton of optional argumnets to this
        # show(nm[i]) 

In [156]:
Q = gap("QuaternionGroup(8)")
subgroupsQ = [H.Representative() for H in Q.ConjugacyClassesSubgroups()]
e = subgroupsQ[0]
C2 = subgroupsQ[1]
I = subgroupsQ[2]
J = subgroupsQ[3]
K = subgroupsQ[4]

print_symbolic_norm(C2,Q)


============ Norm from C2 to Q8 ============

Norm of

	a0 C2/1 + a1 C2/C2

has coefficients:

coefficient of Q8/1:	 a0 * (a0 + a1) * (2*a0^2 + 2*a0*a1 + a1^2)
coefficient of Q8/C2:	 (1/4) * a1 * (a1 + 2) * (a1 - 1)^2
coefficient of Q8/C4:	 (1/2) * (a1 - 1) * a1
coefficient of Q8/C4:	 (1/2) * (a1 - 1) * a1
coefficient of Q8/C4:	 (1/2) * (a1 - 1) * a1
coefficient of Q8/Q8:	 a1
